In [208]:
import pandas as pd 
from datetime import date
import ast
import math

emails_df = pd.read_parquet('/Users/alina/Downloads/Данные Python для отправки студентам/emails.gzip')
salary_df = pd.read_csv('/Users/alina/Downloads/Данные Python для отправки студентам/salary.csv', delimiter=';')

employees_dict = open("/Users/alina/Downloads/Данные Python для отправки студентам/employess_dict.txt", "r")
employees_dict = employees_dict.read()
employees = ast.literal_eval(employees_dict[12:])

In [50]:
salary_df

,ID,DATE,TYPE,VALUE,Unnamed: 4
0,1.0,2019-12-01,salary,50000.0,NaN
1,NaN,NaN,NaN,NaN,NaN
2,1.0,2020-01-01,salary,50000.0,NaN
3,1.0,2020-02-01,salary,50000.0,NaN
4,1.0,2020-03-01,salary,50000.0,NaN
...,...,...,...,...,...
98,4.0,2020-09-01,bonus,7000.0,NaN
99,4.0,2020-10-01,bonus,7000.0,NaN
100,4.0,2020-11-01,bonus,7000.0,NaN
101,4.0,2020-12-01,bonus,7000.0,NaN


In [209]:
def billing_period(start_date, end_date):
    
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    billing_period_days = (end_date - start_date).days
    billing_period_months = math.ceil(billing_period_days/30.4)
    
    return billing_period_months

In [211]:
def avg_salary(database, id, start_date, end_date):

    billing_period_months = billing_period(start_date, end_date)
    
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    

    id_salary = []
    
    for n in database:
        if n[0] == id and n[2] == 'salary':
            id_salary.append(n)

    salary_sum = 0
    
    for n in id_salary:
        if start_date <= datetime.datetime.strptime(n[1], '%Y-%m-%d') <= end_date:
            salary_sum += n[3]
    
    return salary_sum/billing_period_months

In [217]:
def avg_bonus(database, id, start_date, end_date):

    billing_period_months = billing_period(start_date, end_date)

    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    id_salary = []
    
    for n in database:
        if n[0] == id and n[2] == 'bonus':
            id_salary.append(n)

    salary_sum = 0
    
    for n in id_salary:
        if start_date <= datetime.datetime.strptime(n[1], '%Y-%m-%d') <= end_date:
            salary_sum += n[3]
    
    return salary_sum/billing_period_months

In [215]:
avg_salary(salary, 2, '2020-01-01', '2020-12-01')

62416.666666666664

In [213]:
billing_period('2020-01-01', '2020-12-01')

12

In [202]:
start_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2021-01-01', '%Y-%m-%d')

billing_period_days = (end_date - start_date).days
billing_period_months = round(billing_period_days/30.4)

In [203]:
billing_period_months

12

In [218]:
def form_report(employees, salary, emails, start_date, end_date):

    billing_period_months = billing_period(start_date, end_date)
    
    id_list = []
    
    for emp in employees:
        id_list.append(emp[0])
        
    id_set = set(id_list)
    
    fullname_by_id = dict()
    
    for emp in employees:
        fullname_by_id[emp[0]] = ', '.join(emp[1:])
        
    fullname_by_id  = dict(sorted(fullname_by_id.items()))

    emails_by_id = {id: [] for id in id_set}

    for e in emails:
        for id in id_set:
            if int(e[1]) == id and e[2] not in emails_by_id[id]:
                emails_by_id[id].append(e[2])
    
    report_list = []
    for id in id_set:
        report_list.append([])
        report_list[id-1].append(str(id))
        report_list[id-1].extend([fullname_by_id[id],
                                 str(avg_salary(salary, id, start_date, end_date)),
                                 str(avg_bonus(salary, id, start_date, end_date)),
                                 ', '.join(emails_by_id[id])])
        
    for rep in report_list:
        rep = [elem for elem in rep if elem != '']
        print(', '.join(rep))



In [219]:
form_report(employees, salary, emails, '2020-01-01', '2020-12-01')

1, Шершуков, Виктор, Кузьмич, 51750.0, 9583.333333333334, shershuko@mail.ru, shershuko-v@mail.ru
2, Битова, Анастасия, Юрьевна, 62416.666666666664, 8083.333333333333, bitova@mail.ru
3, Кириллов, Валентин, Владиславович, 60666.666666666664, 0.0, kirillov@mail.ru
4, Игнатьев, Игорь, Дмитриевич, 62333.333333333336, 7000.0


В общем случае расчетный период составляет 12 календарных месяцев, предшествующих периоду, когда за сотрудником сохраняется средняя заработная плата (ст. 139 ТК РФ).